In [12]:
import numpy as np
import pandas as pd
import os, gc, cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

ModuleNotFoundError: No module named 'tensorflow'

In [14]:
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow==1.15
ERROR: No matching distribution found for tensorflow==1.15


In [8]:
!pip install opencv-python

In [2]:
train_csv = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../input/planets-dataset/planet/planet/train_classes.csv'

In [3]:
train_csv.head()

In [4]:
train_csv = train_csv.sort_values(by = 'image_name')

In [5]:
train_csv.head()

In [6]:
tags = train_csv['tags'].str.split(' ', expand = True)

In [7]:
all_tags = []
for tag in train_csv['tags']:
    tag_list = tag.split(' ')
    all_tags.extend(tag_list)
    
all_tags = list(set(all_tags))

In [8]:
gc.collect()

In [9]:
all_tags

In [10]:
tag_to_idx = dict(zip(all_tags, range(len(all_tags))))

In [11]:
tag_to_idx

In [12]:
img_file_names = train_csv['image_name'].tolist()

In [13]:
len(img_file_names)

In [14]:
def read_images(fname):
    """ Function to import and typecast image."""
    img = tf.io.read_file('../input/planets-dataset/planet/planet/train-jpg' + '/' + fname + '.jpg')
    img = tf.image.decode_jpeg(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [128, 128])
    return img

In [15]:
del tag_list
del all_tags

In [16]:
gc.collect()

In [ ]:
### Data preprocessing

def preprocess_images(img_file_names):
    train_imgs, train_labels = [], []

    for img_name in img_file_names:
        ix = int(tf.strings.split(img_name, '_')[1])

        train_imgs.append(read_images(img_name))
        labels = np.zeros(shape = [17,])
        tags = train_csv.at[ix, 'tags'].split(' ')
        tags_ix = [tag_to_idx[t] for t in tags]
        labels[tags_ix] = 1

        train_labels.append(labels)
        gc.collect()
    
    return train_imgs, train_labels

train_imgs, train_labels = preprocess_images(img_file_names)

In [ ]:
len(train_imgs)

In [ ]:
len(train_labels)

In [ ]:
train_imgs = np.array(train_imgs)

In [ ]:
train_imgs.shape

In [ ]:
train_labels = np.array(train_labels)
train_labels.shape

In [ ]:
train_imgs = tf.cast(train_imgs, tf.float32)
train_labels = tf.cast(train_labels, tf.float32)

In [ ]:
!ls -l

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))

In [ ]:
dir(dataset)

In [ ]:
### Shuffle and batch dataset
dataset = dataset.shuffle(40479).batch(64)

In [ ]:
gc.collect()

In [ ]:
### Number of batches in dataset
dataset.cardinality().numpy()

In [ ]:
train_ds = dataset.skip(250)
test_ds = dataset.take(250)

In [ ]:
dir(keras.applications)

In [ ]:
from tensorflow.keras.applications.xception import Xception

In [ ]:
base_model = Xception(weights = 'imagenet', include_top = False, input_shape = (128, 128, 3))

In [ ]:
base_model.trainable = False
    
poly_schedule = optimizers.schedule.PolynomialDecay(initial_learning_rate = 3e-4, decay_steps = 10000,
                                                    end_learning_rate = 1e-5, power = 0.5,
                                                    cycle = False)

optimizer = optimizers.Adam(learning_rate = poly_schedule)

callback_list = [callbacks.EarlyStopping(patience = 3, restore_best_weights = True, min_delta = 0.03, )
                ]

final_model = layers.Sequential([keras.preprocessing.experimental.Rescaling(1/255),
                                 base_model,
                                 layers.Flatten(),
                                 layers.Dense(300, activation = tf.nn.selu),
                                 layers.LayerNormalization(),
                                 layers.Dropout(0.5),
                                 layers.Dense(17, activation = tf.nn.sigmoid)
                                ])

In [ ]:
def fbeta_score_K(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return result

In [ ]:
final_model.compile(metrics = [fbeta_score_K], loss = 'binary_crossentropy', optimizer = optimizer)
train_history = final_model.fit(train_ds, epochs = 10, validation_data = test_ds)

In [ ]:
### Use the loss and the number of epochs to plot a graph
train_history.history.keys()